In [19]:
import pandas as pd
df = pd.read_csv("Data_120294_2023-03-29.csv")

/tmp/ipykernel_14910/3493430047.py:2: DtypeWarning: Columns (2,64,91,94,97,106,108,109,110,111,112,114,116,119,120,122,127,139,142,143,144,145,152,153,154,155,156,157,159,160,161,163,164,165,169,170,171,172,175,178,179,180,187,189,215,216,217,218,219,317,318) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data_120294_2023-03-29.csv")


In [20]:
#Save the number of rows
pre_len = len(df.index)
df = df[df['tx_date'].notna()]
post_len = len(df.index)

print(f"Filtered to only transplanted patients: {post_len}/{pre_len} ({post_len/pre_len*100:.2f}% remaining)")

# Compute the time to transplant (waitlist time -> wl_time)
df['wl_time'] = (pd.to_datetime(df['tx_date'], format='%d%b%Y') - pd.to_datetime(df['init_date'], format='%d%b%Y')).dt.days

Filtered to only transplanted patients: 77410/120264 (64.37% remaining)


In [21]:
# Find all non-numerical columns that we need to transform prior to imputation

non_numerical_cols = df.select_dtypes(exclude=['number']).columns
print(f"Non-numerical columns: {len(non_numerical_cols)}")

# Find all the categorical columns by looking at the number of unique values
categorical_cols = [col for col in non_numerical_cols if df[col].nunique() < 10]
print(f"Categorical columns: {len(categorical_cols)}")

non_numerical_non_categorical_cols = [col for col in non_numerical_cols if col not in categorical_cols]
print(f"Non-numerical non-categorical columns: {len(non_numerical_non_categorical_cols)}")

pre_drop_len = len(df.columns)
df = df.drop(categorical_cols, axis=1)
df = df.drop(non_numerical_non_categorical_cols, axis=1)

print(f"Dropped categorical columns ({pre_drop_len} -> {len(df.columns)})")


Non-numerical columns: 76
Categorical columns: 59
Non-numerical non-categorical columns: 17
Dropped categorical columns (321 -> 245)


In [22]:
from missingpy import MissForest

# Remove cols with all rows missing
df = df.dropna(axis=1, how='all')
print(f"Imputing {len(df.columns)} columns")
df = MissForest().fit_transform(df)

print(f"Imputation complete")


Imputing 241 columns


/home/maliti/Prog/time-to-transplantation/.conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/maliti/Prog/time-to-transplantation/.conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


KeyboardInterrupt: 

In [ ]:
df = df.dropna(subset=['wgt_kg_calc', 'age', 'hgt_cm_calc'])
cols_with_missing = [col for col in df.columns if df[col].isnull().any()] # Get columns with missing values
print(f"Imputing {len(cols_with_missing)} columns with {df.isnull().sum().sum()} missing values")

rows_bmi_tcr_missing = df[df['bmi_tcr'].isnull()].index

y = MissForest().fit_transform(df) # Impute
df = pd.DataFrame(y, columns=df.columns)
print(f"Imputed {len(cols_with_missing)} columns, now with {df.isnull().sum().sum()} missing values")

# drop all rows except the ones that had missing values in the bmi_tcr column
df = df.drop(df.index.difference(rows_bmi_tcr_missing))
# compute column "true_bmi"
df['true_bmi'] = df['wgt_kg_calc'] / (df['hgt_cm_calc'] / 100) ** 2
# compute column "bmi_diff"
df['bmi_diff'] = df['bmi_tcr'] - df['true_bmi']
# compute column "bmi_diff_percent"
df['bmi_diff_percent'] = df['bmi_diff'] / df['true_bmi'] * 100
df
